In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np 
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import activations
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation, Add, AveragePooling2D, BatchNormalization, Conv2D, Conv2DTranspose, Dense, Flatten, Input, Reshape, ZeroPadding2D
from tensorflow.keras.applications import ResNet101
from PIL import Image
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
def res_identity(x, filters): 
    # resnet block where dimension doesnot change.
    # The skip connection is just simple identity conncection
    # There will be 3 blocks and then input will be added

    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters
  
    # first block 
    x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # second block # bottleneck (but size kept same with padding)
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # third block activation used after adding the input
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)

    # add the input 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [ ]:
def res_conv(x, s, filters):
    # here the input size changes
    x_skip = x
    f1, f2 = filters

    # first block
    x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
    # when s = 2 then it is like downsizing the feature map
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # second block
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # third block
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)

    # shortcut 
    x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [ ]:
def dres_conv(x, s, filters):
    # here the input size changes
    x_skip = x
    f1, f2 = filters

    # third block
    x = Conv2DTranspose(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # second block
    x = Conv2DTranspose(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # third block
    x = Conv2DTranspose(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
    # when s = 2 then it is like downsizing the feature map
    x = BatchNormalization()(x)

    # shortcut 
    x_skip = Conv2DTranspose(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [ ]:
def dres_identity(x, filters): 
    # resnet block where dimension doesnot change.
    # The skip connection is just simple identity conncection
    # There will be 3 blocks and then input will be added

    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters

    # first block 
    x = Conv2DTranspose(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # second block # bottleneck (but size kept same with padding)
    x = Conv2DTranspose(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = Activation(activations.relu)(x)
    x = BatchNormalization()(x)

    # third block activation used after adding the input
    x = Conv2DTranspose(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)

    # add the input 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [ ]:
def Models():

    # Encoder Part
    input_im = Input(shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))
    Encoder = ResNet101(include_top=False, weights='imagenet', input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))
    x = Encoder(input_im)
    x = Flatten()(x)

    encoding = Dense(2048, kernel_initializer='he_normal')(x)

    x = Dense(2 * 2 * 2048, kernel_initializer='he_normal')(encoding)
    x = Reshape((2, 2, 2048))(x)

    x = dres_conv(x, s=2, filters=(512, 2048))
    x = dres_identity(x, filters=(512, 2048))
    x = dres_identity(x, filters=(512, 2048))

    x = dres_conv(x, s=2, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))
    x = dres_identity(x, filters=(256, 1024))


    x = dres_conv(x, s=2, filters=(128, 512))
    x = dres_identity(x, filters=(128, 512))
    x = dres_identity(x, filters=(128, 512))
    x = dres_identity(x, filters=(128, 512))

    x = dres_conv(x, s=1, filters=(64, 256))
    x = dres_identity(x, filters=(64, 256))
    x = dres_identity(x, filters=(64, 256))
    # print(x.shape)
    decoded = Conv2DTranspose(3, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.sigmoid)(x)

    encoder = tf.keras.Model(inputs=input_im, outputs=encoding, name='Encoder')
    # decoder = tf.keras.Model(inputs=Input(encoding), outputs=decoded, name='Decoder')
    ae = tf.keras.Model(inputs=input_im, outputs=decoded, name='Auto_Encoder')


    return ae, encoder

In [ ]:
from keras.datasets import cifar10
import numpy as np
(x_train, _), (x_test, _) = cifar10.load_data()

Using TensorFlow backend.


In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
auto_encoder, encoder = Models()
auto_encoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
auto_encoder.summary()

Model: "Auto_Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
resnet101 (Model)               (None, 1, 1, 2048)   42658176    input_1[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           resnet101[1][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 2048)         4196352     flatten[0][0]                    
_______________________________________________________________________________________

In [ ]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
auto_encoder.fit(x_train, x_train, epochs=25, batch_size=128, shuffle=True, validation_data=(x_test, x_test), callbacks=tensorboard_callback)

Epoch 1/50
391/391 [==============================] - 965s 2s/step - loss: 18.0673 - val_loss: 8.8613
Epoch 2/50
391/391 [==============================] - 946s 2s/step - loss: 5.7549 - val_loss: 3.5976
Epoch 3/50
391/391 [==============================] - 948s 2s/step - loss: 2.3721 - val_loss: 1.5453
Epoch 4/50
391/391 [==============================] - 949s 2s/step - loss: 0.9839 - val_loss: 0.6351
Epoch 5/50
391/391 [==============================] - 948s 2s/step - loss: 0.4068 - val_loss: 0.2743
Epoch 6/50
391/391 [==============================] - 948s 2s/step - loss: 0.1722 - val_loss: 0.1321
Epoch 7/50
391/391 [==============================] - 947s 2s/step - loss: 0.1129 - val_loss: 166663.0312
Epoch 8/50
391/391 [==============================] - 947s 2s/step - loss: 0.0663 - val_loss: 0.0690
Epoch 9/50
391/391 [==============================] - 947s 2s/step - loss: 0.0355 - val_loss: 0.0359
Epoch 10/50
391/391 [==============================] - 947s 2s/step - loss: 0.0296 - 

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# plt.plot(history.history["accuracy"])
# plt.plot(history.history["val_accuracy"])
# plt.title("Model Accuracy")
# plt.ylabel("Accuracy")
# plt.xlabel("Epoch")
# plt.legend(["train", "test"], loc="upper left")
# plt.save("/content/gdrive/My Drive/Colab/ResNet/ResNet101_ModelLoss.jpeg")
# plt.show()

In [ ]:
# plt.plot(history.history["loss"])
# plt.plot(history.history["val_loss"])
# plt.title("Model Loss")
# plt.ylabel("Loss")
# plt.xlabel("Epoch")
# plt.legend(["train", "test"], loc="upper left")
# plt.save("/content/gdrive/My Drive/Colab/ResNet/ResNet50_ModelLoss.jpeg")
# plt.show()

In [ ]:
MODEL_STORE_PATH = "/content/gdrive/My Drive/Colab/TF_ResNet101.h5"
auto_encoder.save(MODEL_STORE_PATH)